In [2]:
from SBox import *

In [9]:
# LAT-profile

def gen_lat_profile(self): 
    lat = [[0 for _ in range(2 ** self.output_len)] for _ in range(2 ** self.input_len)]

    for input_mask in range(2 ** self.input_len):
        for output_mask in range(2 ** self.output_len):
            count = 0
            for i in range(2 ** self.input_len):
                input_value = i & input_mask
                output_value = self.sbox[i] & output_mask
                if (bin(input_value).count('1') + bin(output_value).count('1')) % 2 == 0:
                    count += 1

            lat[input_mask][output_mask] = count - (2 ** (self.input_len - 1))

    return lat
    
SBox.gen_lat_profile = gen_lat_profile

print(aes_sbox.gen_lat_profile())


[[128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 12, 0, 12, 14, 6, 2, -6, 12, 0, 8, 12, 2, 10, 2, -6, 8, -12, 12, -8, -10, -2, -10, -2, -8, 12, 0, 4, -10, 14, -6, 2, -4, -4, -4, -4, 10, -2, 6, 10, -12, 12, -8, 0, 2, 6, 2, 6, 8, 8, -12, -12, -10, -6, 6, -6, 12, 4, 4, 12, -

In [8]:
# LAT - path

sbox = aes_sbox.sbox

def my_round(x, k):
    # rotate x by one bit
    bitlen = x.bit_length()
    y = x >> 1
    y0 = (x % 2) << (bitlen - 1)
    y = y | y0

    # add key
    y = y ^ k

    # run the sbox
    y = sbox[y]

    return y

def my_cipher(x, k):
    x = x ^ k
    k1 = sbox[k]
    x = my_round(x, k1)
    k2 = sbox[k1]
    x = my_round(x, k2)
    return x

def gen_lat_profile():
    input_len = 8
    output_len = 8

    lat = [[0 for _ in range(2 ** output_len)] for _ in range(2 ** input_len)]

    for input_mask in range(2 ** input_len):
        for output_mask in range(2 ** output_len):
            count = 0
            for i in range(2 ** input_len):
                input_value = i & input_mask
                output_value = sbox[i] & output_mask
                if (bin(input_value).count('1') + bin(output_value).count('1')) % 2 == 0:
                    count += 1

            lat[input_mask][output_mask] = count - (2 ** (input_len - 1))

    return lat

lat = gen_lat_profile()

max_diff = 0
max_x = 0
max_y = 0

for x in range(2 ** 8):
    for y in range(2 ** 8):
        if x == 0 and y == 0:
            continue
        diff = lat[x][y]
        if abs(diff) > max_diff:
            max_diff = diff
            max_x = x
            max_y = y

print(max_diff, bin(max_x), bin(max_y))

# Otrzymujemy 
# max_diff = 16
# max_x = 0b00000001
# max_y = 0b10001000
# czyli ścieżka linowa z x = 0b00000001 do y = 0b10001000 zachodzi z prawdopodobieństwem 1/2 + 16/128 = 1/2 + 1/8 = 5/8
# odpowiada to aproksymacji liniowej
# x_0 = y_7 + y_3


16 1 136
0b1 0b10001000
